In [2]:
import numpy as np
import pandas as pd
import datetime
import os

In [3]:
# Replace 'your_directory_path' with the path to your folder
folder_path = 'Runner_final'

# List all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

runners_final_df = pd.DataFrame()

for file in csv_files:
    df = pd.read_csv(f'Runner_final/{file}')
        
    runners_final_df = pd.concat([runners_final_df, df])
    

In [4]:
# Replace 'your_directory_path' with the path to your folder
folder_path = 'catchers'

# List all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

catchers_final_df = pd.DataFrame()

for file in csv_files:
    df = pd.read_csv(f'catchers/{file}')
        
    catchers_final_df = pd.concat([catchers_final_df, df])
    

In [5]:
catchers_final_df.replace(' --', np.nan, inplace=True)

In [6]:
catchers_final_df['Arm_Strength'] = catchers_final_df['Arm_Strength'].astype(float)

catchers_final_df['Exchange'] = catchers_final_df['Exchange'].astype(float)

In [7]:
runners_final_df.replace(' --', np.nan, inplace=True)

In [8]:
stealing_df = pd.read_csv('combined.csv')

stealing_df.drop(columns=['Unnamed: 0'], inplace=True)

stealing_df.head()

,Start_Base,End_Base,Event,Respon_Pitcher,runner_id,runner_name,pitcher_id,pitcher_name,batter_id,batter_name,pitch_hand,credit,credit_id,homeScore,awayScore,inning,isTopInning,Date,Action,Base
0,1B,NaN,Caught Stealing 2B,NaN,545341,Randal Grichuk,643338,Chad Green,543760,Marcus Semien,R,Gary Sánchez,596142,2,2,7,True,2021-04-01,Caught,2B
1,1B,2B,Stolen Base 2B,NaN,643565,Mike Tauchman,605447,Jordan Romano,457803,Jay Bruce,R,Danny Jansen,643376,2,2,9,False,2021-04-01,Stolen,2B
2,2B,3B,Stolen Base 3B,NaN,643565,Mike Tauchman,605447,Jordan Romano,640449,Clint Frazier,R,Danny Jansen,643376,2,2,9,False,2021-04-01,Stolen,3B
3,1B,2B,Stolen Base 2B,NaN,666185,Dylan Carlson,489334,Craig Stammen,425877,Yadier Molina,R,Austin Nola,543592,3,6,4,True,2020-09-30,Stolen,2B
4,2B,3B,Stolen Base 3B,NaN,502054,Tommy Pham,650893,Génesis Cabrera,543592,Austin Nola,L,Yadier Molina,425877,4,6,6,False,2020-09-30,Stolen,3B


In [9]:
stealing_df['Result'] = stealing_df['Action'].apply(lambda x : 1 if x =='Stolen' else 0)

In [19]:
data_dict = []

final_df = {'Result':[], 'Exchange':[] ,'Arm_Strength':[], 'Sprint':[], 'Date': [], 'runner_id': [], 'runner_name': []}

for row, j in stealing_df.iterrows():
    catcher_id = j['credit_id']
    
    runner_id = j['runner_id']
    
    catcher_name = j['credit']
    
    runner_name = j['runner_name']
    
    date = j['Date']
    
    if len(runners_final_df.loc[(runners_final_df['runner_name'] == runner_name) & (runners_final_df['Date'] == date)]) == 0:
        continue
    
    runner_sprint = runners_final_df.loc[(runners_final_df['runner_name'] == runner_name) & (runners_final_df['Date'] == date)]['Sprint'].tolist()[0]
    
    try:

        catcher_df = pd.read_csv(f'catchers/{catcher_name}.csv')
        
        
    except:
        print(catcher_name)
        continue
        
    if 'Exchange' not in catcher_df.columns:
        continue
        
    catcher_df['Result'] = catcher_df['Action'].apply(lambda x : 1 if x =='Stolen' else 0)
        
    catcher_df.drop(columns=['Unnamed: 0'], inplace=True)

    catcher_df.drop_duplicates(inplace=True)

    catcher_past = catcher_df.loc[catcher_df['Date'] < date].sort_values(by='Date').tail(5)

    if len(catcher_past) < 5:
        continue

    catcher_past['Result'] = catcher_past['Action'].apply(lambda x : 1 if x =='Stolen' else 0)
    
    uniques = catcher_past['Arm_Strength'].unique()




    if catcher_past['Arm_Strength'].dtypes == 'object':
    
        arm = catcher_past['Arm_Strength'].tolist()
        numbers = [item.strip() for item in arm if item != ' --']

        if len(numbers) == 0:
            catcher_past['Arm_Strength'].replace(' --', np.nan, inplace=True)
            catcher_past['Arm_Strength'] = catcher_past['Arm_Strength'].astype(float)
            
        else:
            
            repl = float(numbers[0])
            catcher_past['Arm_Strength'].replace(' --', repl, inplace=True)
            catcher_past['Arm_Strength'] = catcher_past['Arm_Strength'].astype(float)

        
    if catcher_past['Exchange'].dtypes == 'object':
    
        arm = catcher_past['Exchange'].tolist()
        numbers = [item.strip() for item in arm if item != ' --']
        if len(numbers) == 0:
            catcher_past['Exchange'].replace(' --', np.nan, inplace=True)
            catcher_past['Exchange'] = catcher_past['Exchange'].astype(float)
            
        else:
            
            repl = float(numbers[0])
            catcher_past['Exchange'].replace(' --', repl, inplace=True)
            catcher_past['Exchange'] = catcher_past['Exchange'].astype(float)

    sprints = []

    for row, i in catcher_past.iterrows():

        runner = i['runner_name']

        current_year = i['Year'] 
        
        if len(runners_final_df.loc[(runners_final_df['runner_name'] == runner) & (runners_final_df['Current_Year'] == current_year)]['Sprint']) == 0:
            sprints.append(np.nan)
            continue

        sprint = runners_final_df.loc[(runners_final_df['runner_name'] == runner) & (runners_final_df['Current_Year'] == current_year)]['Sprint'].tolist()[0]

        sprints.append(sprint)
        
    catcher_past['Sprint'] = sprints
    
    if catcher_past['Sprint'].dtypes == 'object':
    
        arm = catcher_past['Sprint'].tolist()
        numbers = [item.strip() for item in arm if item != np.nan]
        if len(numbers) == 0:
            catcher_past['Sprint'].replace(' --', np.nan, inplace=True)
            catcher_past['Sprint'] = catcher_past['Sprint'].astype(float)
            
        else:
            
            repl = float(numbers[0])
            catcher_past['Sprint'].replace(np.nan, repl, inplace=True)
            catcher_past['Sprint'] = catcher_past['Sprint'].astype(float)

    past_data = catcher_past[['Result', 'Exchange' ,'Arm_Strength', 'Sprint', 'Date']].sort_values(by='Date', ascending=False)

    lag = 1

    cols = ['Result', 'Exchange' ,'Arm_Strength', 'Sprint']

    for row, i in past_data.iterrows():

        for col in cols:

            if f'{col}_{lag}' not in final_df:
                final_df[f'{col}_{lag}'] = []

            final_df[f'{col}_{lag}'].append(i[col])

        lag += 1
    
    Exchange = catcher_df.loc[catcher_df['Date'] < date]['Exchange'].tolist()[0]
    
    Result = catcher_df.loc[catcher_df['Date'] < date]['Result'].tolist()[0]
    
    Arm_Strength = catcher_df.loc[catcher_df['Date'] < date]['Arm_Strength'].tolist()[0]
    
    final_df['Result'].append(Result)
    final_df['Exchange'].append(Exchange)
    final_df['Arm_Strength'].append(Arm_Strength)
    final_df['Sprint'].append(runner_sprint)
    final_df['Date'].append(date)
    final_df['runner_id'].append(runner_id)
    final_df['runner_name'].append(runner_name)
    

In [20]:
stolen_df = pd.DataFrame(final_df)
stolen_df.to_csv('stolen_df_2.csv', index=False)